<img src="https://static.javatpoint.com/tutorial/machine-learning/images/hyperparameters-in-machine-learning2.png" style="width:400px;height:200px;">
<h1 color="blue">Hyperparameter Tunning</h1>


Hyperparameter tuning, also known as hyperparameter optimization or simply "hyperparameter search," is the process of finding the best set of hyperparameters for a machine learning model to optimize its performance on a specific task or dataset. Hyperparameters are parameters that are not learned from the data but are set prior to training the model. Examples of hyperparameters include learning rate, the number of hidden layers and units in a neural network, the depth of a decision tree, or the regularization strength in a support vector machine.

In [134]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

In [135]:
iris = load_iris()

In [136]:
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [137]:
iris.data[1:10]

array([[4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

In [138]:
df = pd.DataFrame(iris.data,columns=iris.feature_names)

In [139]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [144]:
df['target'] = iris.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,flower
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [145]:
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x :iris.target_names[x])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,flower
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [146]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target,test_size= 0.3)

In [147]:
from sklearn.svm import SVC

In [148]:
model = SVC(kernel = 'rbf',C = 30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test,y_test)

1.0

In [149]:
def get_score(model,x_train,x_test,y_train,y_test):
    model.fit(x_train,y_train)
    return model.score(x_test,y_test)

In [150]:
score_1 = []

In [153]:
from sklearn.model_selection import KFold
kf= KFold(n_splits=6)
kf

KFold(n_splits=6, random_state=None, shuffle=False)

In [158]:
for train_index,test_index in kf.split(iris.data):
    x_train,x_test,y_train,y_test= iris.data[train_index],iris.data[test_index],iris.target[train_index],iris.target[test_index]
    score_1.append(get_score(SVC(kernel = 'rbf',C = 30,gamma='auto'),x_train,x_test,y_train,y_test))

In [159]:
score_1

[1.0, 1.0, 0.88, 0.92, 0.92, 0.84, 1.0, 1.0, 0.88, 0.92, 0.92, 0.84]

In [161]:
from sklearn.model_selection import cross_val_score   #5 folods
cross_val_score(SVC(kernel = 'rbf',C = 30,gamma='auto'),iris.data,iris.target)

array([0.96666667, 1.        , 0.9       , 0.93333333, 1.        ])

In [163]:
cross_val_score(SVC(kernel = 'rbf',C =10,gamma='auto'),iris.data,iris.target)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [164]:
cross_val_score(SVC(kernel = 'linear',C =10,gamma='auto'),iris.data,iris.target)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [172]:
#same processs using for loop
import numpy as np
kernels = ['rbf','linear']
avg_scores={}
C = [30,10,1]
for kval in kernels:
    for cval in C:
        cv_score =cross_val_score(SVC(kernel = 'rbf',C =10,gamma='auto'),iris.data,iris.target)
        avg_scores[kval +'_'+str(cval)] = np.average(cv_score)
        
avg_scores

{'rbf_30': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_1': 0.9800000000000001,
 'linear_30': 0.9800000000000001,
 'linear_10': 0.9800000000000001,
 'linear_1': 0.9800000000000001}

In [177]:
#gridsearchCV DOES THE SAME loop iterations as above
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(SVC(gamma='auto'),{
    'C':[1,10,20,30,70,90],
    'kernel':['linear','rbf']
},cv= 5,return_train_score =False)

clf.fit(iris.data,iris.target)
clf.cv_results_

{'mean_fit_time': array([2.01516151e-03, 1.18150711e-03, 0.00000000e+00, 1.60665512e-03,
        1.61075592e-03, 9.04560089e-05, 1.20925903e-03, 0.00000000e+00,
        3.17049026e-04, 1.10602379e-03, 1.02281570e-04, 0.00000000e+00]),
 'std_fit_time': array([0.0022308 , 0.00101853, 0.        , 0.00321331, 0.00322151,
        0.00018091, 0.00241852, 0.        , 0.00041931, 0.00221205,
        0.00020456, 0.        ]),
 'mean_score_time': array([0.00215354, 0.00254073, 0.00110836, 0.        , 0.00146008,
        0.        , 0.        , 0.00172706, 0.00167727, 0.00023494,
        0.00160837, 0.0013083 ]),
 'std_score_time': array([0.00294453, 0.00233875, 0.00221672, 0.        , 0.00292015,
        0.        , 0.        , 0.00345411, 0.00285538, 0.00046988,
        0.00321674, 0.0026166 ]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20, 30, 30, 70, 70, 90, 90],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],

In [178]:
#hyperparameter tunning is the optimal way of coosing the parameters

In [180]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002015,0.002231,0.002154,0.002945,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001182,0.001019,0.002541,0.002339,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000000,0.000000,0.001108,0.002217,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.966667,1.0,0.973333,0.038873,4
3,0.001607,0.003213,0.000000,0.000000,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
4,0.001611,0.003222,0.001460,0.002920,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,6
5,0.000090,0.000181,0.000000,0.000000,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.000000,0.900000,0.966667,1.0,0.966667,0.036515,5
6,0.001209,0.002419,0.000000,0.000000,30,linear,"{'C': 30, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.900000,1.0,0.960000,0.048990,9
7,0.000000,0.000000,0.001727,0.003454,30,rbf,"{'C': 30, 'kernel': 'rbf'}",0.966667,1.000000,0.900000,0.933333,1.0,0.960000,0.038873,9
8,0.000317,0.000419,0.001677,0.002855,70,linear,"{'C': 70, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,6
9,0.001106,0.002212,0.000235,0.000470,70,rbf,"{'C': 70, 'kernel': 'rbf'}",1.000000,0.966667,0.900000,0.933333,1.0,0.960000,0.038873,9


In [184]:
df.drop([
'mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time'],axis = 'columns',inplace = True)

In [185]:
df

,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
1,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
2,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.966667,1.0,0.973333,0.038873,4
3,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
4,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,6
5,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.000000,0.900000,0.966667,1.0,0.966667,0.036515,5
6,30,linear,"{'C': 30, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.900000,1.0,0.960000,0.048990,9
7,30,rbf,"{'C': 30, 'kernel': 'rbf'}",0.966667,1.000000,0.900000,0.933333,1.0,0.960000,0.038873,9
8,70,linear,"{'C': 70, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,6
9,70,rbf,"{'C': 70, 'kernel': 'rbf'}",1.000000,0.966667,0.900000,0.933333,1.0,0.960000,0.038873,9


In [186]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.980000
1,1,rbf,0.980000
2,10,linear,0.973333
3,10,rbf,0.980000
4,20,linear,0.966667
5,20,rbf,0.966667
6,30,linear,0.960000
7,30,rbf,0.960000
8,70,linear,0.966667
9,70,rbf,0.960000


In [187]:
dir(clf)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 '

In [188]:
clf.best_score_

0.9800000000000001

In [189]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [194]:
# to compute the loop from 1 to 5 it may take high comutational power to over come that we use randamonizesearchcv
#won't iterate the all the parameters but iterates random parameters
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(SVC(gamma= 'auto'),{
    'C':[1,10,20,30],
    'kernel':['linear','rbf']
},cv=5,return_train_score=False,n_iter=2) #only trys 2 combinations

rs.fit(iris.data,iris.target)

RandomizedSearchCV(cv=5, estimator=SVC(gamma='auto'), n_iter=2,
                   param_distributions={'C': [1, 10, 20, 30],
                                        'kernel': ['linear', 'rbf']})

In [195]:
rs.cv_results_

{'mean_fit_time': array([0.003443  , 0.00252156]),
 'std_fit_time': array([0.00199468, 0.00294789]),
 'mean_score_time': array([0.00060496, 0.00010953]),
 'std_score_time': array([0.0008046 , 0.00021906]),
 'param_kernel': masked_array(data=['linear', 'linear'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_C': masked_array(data=[1, 10],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'kernel': 'linear', 'C': 1}, {'kernel': 'linear', 'C': 10}],
 'split0_test_score': array([0.96666667, 1.        ]),
 'split1_test_score': array([1., 1.]),
 'split2_test_score': array([0.96666667, 0.9       ]),
 'split3_test_score': array([0.96666667, 0.96666667]),
 'split4_test_score': array([1., 1.]),
 'mean_test_score': array([0.98      , 0.97333333]),
 'std_test_score': array([0.01632993, 0.03887301]),
 'rank_test_score': array([1, 2])}

In [197]:
df = pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

In [198]:
df

,param_C,param_kernel,mean_test_score
0,1,linear,0.980000
1,10,linear,0.973333


In [216]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [223]:
model_params = {
    'svm':{
        'model':SVC(gamma='auto'),
        'params':{
            'C': [1,20,10,30],
            'kernel':['rbf','linear']
        }
    },
    'random_forect':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators':[1,5,10]
        }
    },
    'logistic_regression':{
        'model': LogisticRegression(solver = 'liblinear',multi_class='auto'),
        'params': {
            'C':[1,3,9]
        }
    },
    'naive_bayes_gaussian':{
        'model':GaussianNB(),
        'params':{
        }
    },
    'naive_bayes_multinomial':{
        'model':MultinomialNB(),
        'params':{
            
        }
    },
    'decision_tree':{
        'model':DecisionTreeClassifier(),
        'params':{
            'criterion':['gini','entropy']
        }
    }
}

In [224]:
scores = []

for model_name,mp in model_params.items():
    clf=GridSearchCV(mp['model'],mp['params'],cv= 5,return_train_score = False)
    clf.fit(iris.data,iris.target)
    scores.append({
        'model':model_name,
        'best_params':clf.best_params_,
        'best_score':clf.best_score_
    })

In [225]:
scores

[{'model': 'svm',
  'best_params': {'C': 1, 'kernel': 'rbf'},
  'best_score': 0.9800000000000001},
 {'model': 'random_forect',
  'best_params': {'n_estimators': 5},
  'best_score': 0.9466666666666667},
 {'model': 'logistic_regression',
  'best_params': {'C': 9},
  'best_score': 0.9666666666666668},
 {'model': 'naive_bayes_gaussian',
  'best_params': {},
  'best_score': 0.9533333333333334},
 {'model': 'naive_bayes_multinomial',
  'best_params': {},
  'best_score': 0.9533333333333334},
 {'model': 'decision_tree',
  'best_params': {'criterion': 'gini'},
  'best_score': 0.9666666666666668}]

In [226]:
df= pd.DataFrame(scores,columns =['model','best_params','best_score'])

In [227]:
df

,model,best_params,best_score
0,svm,"{'C': 1, 'kernel': 'rbf'}",0.980000
1,random_forect,{'n_estimators': 5},0.946667
2,logistic_regression,{'C': 9},0.966667
3,naive_bayes_gaussian,{},0.953333
4,naive_bayes_multinomial,{},0.953333
5,decision_tree,{'criterion': 'gini'},0.966667
